### The goal of this project, is to develop a recommender system utilizing the Spotify API, where we're gonna consume the data in real time utilizing Spark Streaming and Kafka stored in a docker to process the data.
### This project is going to be separated in 2 parts. The first part gonna be with the producer of Data to simulate a real time data streaming utilizing a Dataset from de following Link:https://research.atspotify.com/datasets/
###  The second part gonna have the creation of Spark session and needed pre-process to train the Content-Based Recommender System.


# First, lets build the Docker using docker compose 

# Step1 : Create a file called docker-compose.yaml in your project directory and paste the following command: 

version: '3'

services:
  
  zookeeper:
    image: wurstmeister/zookeeper
    container_name : zookeeper
    ports:
      - "2181:2181"

  kafka:
    image: wurstmeister/kafka
    container_name : kafka
    ports:
      - "9092:9092"
    environment:
      KAFKA_ADVERTISED_HOST_NAME: localhost
      KAFKA_ZOOKEEPER_CONNECT: zookeeper:2181
      KAFKA_CREATE_TOPICS: "playlist"
    depends_on:
      - zookeeper
      
# Step2: Create docker with docker-compose:
### Open the command prompt, browse into the folder where the project files are placed and execute the command below:
## docker-compose up -d
      
# Step3: Creating kafka topic

### Access the Kafka container (you can do that using Docker Desktop) and execute the following commands below to create and visualize the topic on Kafka:

bash

cd /opt/kafka/bin

### Create topic:
kafka-topics.sh --create --zookeeper zookeeper:2181 --replication-factor 1 --partitions 1 --topic playlist


### List the created topics:
kafka-topics.sh --list --zookeeper zookeeper:2181


### Details about the topic:
kafka-topics.sh --describe --zookeeper zookeeper:2181

In [1]:
import os 
import time 
import kafka
import numpy as np 
import pandas as pd 
from json import dumps 
from kafka import KafkaProducer
import warnings
warnings.filterwarnings('ignore')

# Constructing the Producer

In [2]:
# Kafka host address
SERVER = 'localhost:9092'

In [3]:
#Topic name 
TOPIC = 'playlist'

In [4]:
#Loading set of music data
playlist = pd.read_csv('dados\dataset.csv')

In [5]:
#Adjusting format of 3 important columns: order_id, Artist Name(s) e Artist IDs
playlist['order_id'] = np.arange(len(playlist))
playlist['Artist Name(s)'] = playlist['Artist Name(s)'].str.replace('[^a-zA-Z]','')
playlist['Artist IDs'] = playlist['Artist IDs'].str.replace('[^a-zA-Z]','')

In [6]:
playlist.head(10)

,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,...,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,order_id
0,22a0Ji6EQKkY0tBohlN4Od,7qL7yYYh4SlsjwymwVK0wW,There You Are,There You Are,Kirsten Ludwig,2018-08-06,231240,2,spotify:user:predict0,2018-08-28T19:51:58Z,...,-5.596,0,0.0304,0.334000,0.282000,0.1050,0.316,129.856,4,0
1,4J39ZEbwqHwtWLImUKmrn9,"5CRfAxYjJsDBH5wvWF2nja,55RRRPXwFwQmoTNqN4HBGU",88 Days,Heat,"Sara King,Ian Olney",2018-08-04,227961,8,spotify:user:predict0,2018-08-28T19:51:58Z,...,-10.749,1,0.0333,0.134000,0.582000,0.1340,0.233,155.062,4,1
2,0a12d4HUjOmQSqHqLopWYx,0hy4t2HTGT4flktWAhKcxQ,Castaway,Castaway,ARZLEE,2018-08-10,230000,0,spotify:user:predict0,2018-08-28T19:51:58Z,...,-11.290,1,0.0314,0.110000,0.000032,0.1190,0.290,83.988,4,2
3,4u1DykFW1HjYAGNoDCiXfC,"6Wjyo1J1HR7HlT2bUTZ82T,021wqpA4geq0mtJlARXjon",Arouse,Arouse,"Shagabond,goodboy noah",2018-08-03,213913,30,spotify:user:predict0,2018-08-28T19:51:58Z,...,-6.066,1,0.4330,0.072800,0.000000,0.3680,0.533,91.961,4,3
4,0u7JZm9ORerlZnnxxSdMwl,3Ad8KmjgFzpcTvmVf69GwR,Lonely,Lonely,Hayleau,2018-08-10,258738,21,spotify:user:predict0,2018-08-28T19:51:58Z,...,-3.921,0,0.0406,0.016900,0.000630,0.0542,0.577,98.954,4,4
5,0wuy2BYIVLbflFDqnR9Jay,5kCw0rYUFSJCubb6b2nZrE,Orsay,Strange Affairs,The Svens,2018-08-03,413658,6,spotify:user:predict0,2018-08-28T19:51:58Z,...,-11.858,1,0.0316,0.048600,0.886000,0.1280,0.283,122.992,4,5
6,6LkIZZRrPQIbHMyBR5mTc2,4TOsWuafqeWtrvYXqbnYAV,Nurture,Comrade,Island Fox,2018-08-09,191641,0,spotify:user:predict0,2018-08-28T19:51:58Z,...,-11.415,0,0.0504,0.015900,0.639000,0.1810,0.266,133.925,4,6
7,5U27fxNSd27XtX876xUsfV,"6H2sKUE9xgNcRJo6jP9mBc,4N7qzgwpvz213edAIjuDBM",Dinosaur Hair - Remix,Dinosaur Hair,"Andy Ferro,Charlie Conway",2018-08-10,257152,1,spotify:user:predict0,2018-08-28T19:51:58Z,...,-10.086,1,0.0383,0.456000,0.920000,0.1350,0.663,125.908,4,7
8,5ogJOpmyDsvrAdttU6JLnN,5g4slbnQQL1L9cNzfjnxQY,Breathing Underwater,Long Way Down,Morning Wars,2018-08-03,174999,0,spotify:user:predict0,2018-08-28T19:51:58Z,...,-11.258,0,0.0461,0.000005,0.020400,0.1150,0.477,150.042,4,8
9,65rLHt6A58MFRxlNWVDU1Z,09Wl9YiRr5l1rChWktQD4o,Summer,Summer,NoSo,2018-08-01,232746,22,spotify:user:predict0,2018-08-28T19:51:58Z,...,-7.517,1,0.0380,0.016800,0.007230,0.0706,0.210,123.962,4,9


In [7]:
playlist.columns

Index(['Spotify ID', 'Artist IDs', 'Track Name', 'Album Name',
       'Artist Name(s)', 'Release Date', 'Duration (ms)', 'Popularity',
       'Added By', 'Added At', 'Genres', 'Danceability', 'Energy', 'Key',
       'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness',
       'Liveness', 'Valence', 'Tempo', 'Time Signature', 'order_id'],
      dtype='object')

In [8]:
#Converting the dataframe in a dictionary of musics
#Kafka expects the data structure to be as key and value.
dict_music = playlist.to_dict(orient="records")

In [9]:
dict_music[0:1]

[{'Spotify ID': '22a0Ji6EQKkY0tBohlN4Od',
  'Artist IDs': '7qL7yYYh4SlsjwymwVK0wW',
  'Track Name': 'There You Are',
  'Album Name': 'There You Are',
  'Artist Name(s)': 'Kirsten Ludwig',
  'Release Date': '2018-08-06',
  'Duration (ms)': 231240,
  'Popularity': 2,
  'Added By': 'spotify:user:predict0',
  'Added At': '2018-08-28T19:51:58Z',
  'Genres': 'experimental folk',
  'Danceability': 0.487,
  'Energy': 0.707,
  'Key': 9,
  'Loudness': -5.596,
  'Mode': 0,
  'Speechiness': 0.0304,
  'Acousticness': 0.334,
  'Instrumentalness': 0.282,
  'Liveness': 0.105,
  'Valence': 0.316,
  'Tempo': 129.856,
  'Time Signature': 4,
  'order_id': 0}]

In [10]:
#Kafka producer
if __name__ == "__main__":
    #Create producer 
    producer = KafkaProducer(bootstrap_servers = SERVER,
                            value_serializer = lambda x: x.encode('utf-8'))
    
    #Loop on the music dictionary
    for music in dict_music:
        #Create a list that will be sent to kafka 
        sending = []
        
        #append each column 
        sending.append(music["order_id"])
        sending.append(music["Spotify ID"])
        sending.append(music["Track Name"])
        sending.append(music["Genres"])
        sending.append(music["Popularity"])
        sending.append(music["Duration (ms)"])
        sending.append(music["Artist Name(s)"])
        sending.append(music["Artist IDs"])
        sending.append(music["Release Date"])
        sending.append(music["Danceability"])
        sending.append(music["Energy"])
        sending.append(music["Key"])
        sending.append(music["Loudness"])
        sending.append(music["Mode"])
        sending.append(music["Speechiness"])
        sending.append(music["Acousticness"])
        sending.append(music["Instrumentalness"])
        sending.append(music["Liveness"])
        sending.append(music["Valence"])
        sending.append(music["Tempo"])
        sending.append(music["Time Signature"])
        
        #Put it all together 
        music = ','.join(str(v) for v in sending)
        
        #Send data to the kafka topic 
        print("Next music:")
        print(music)
        producer.send(TOPIC, music)
        time.sleep(0.6)
    
    print("Concluded")

Next music:
0,22a0Ji6EQKkY0tBohlN4Od,There You Are,experimental folk,2,231240,Kirsten Ludwig,7qL7yYYh4SlsjwymwVK0wW,2018-08-06,0.487,0.707,9,-5.596,0,0.0304,0.334,0.282,0.105,0.316,129.856,4
Next music:
1,4J39ZEbwqHwtWLImUKmrn9,88 Days,bedroom pop,8,227961,Sara King,Ian Olney,5CRfAxYjJsDBH5wvWF2nja,55RRRPXwFwQmoTNqN4HBGU,2018-08-04,0.335,0.401,3,-10.749,1,0.0333,0.134,0.582,0.134,0.233,155.062,4
Next music:
2,0a12d4HUjOmQSqHqLopWYx,Castaway,nan,0,230000,ARZLEE,0hy4t2HTGT4flktWAhKcxQ,2018-08-10,0.553,0.422,1,-11.29,1,0.0314,0.11,3.25e-05,0.119,0.29,83.988,4
Next music:
3,4u1DykFW1HjYAGNoDCiXfC,Arouse,alternative r&b,indie r&b,30,213913,Shagabond,goodboy noah,6Wjyo1J1HR7HlT2bUTZ82T,021wqpA4geq0mtJlARXjon,2018-08-03,0.67,0.751,1,-6.066,1,0.433,0.0728,0.0,0.368,0.533,91.961,4
Next music:
4,0u7JZm9ORerlZnnxxSdMwl,Lonely,nan,21,258738,Hayleau,3Ad8KmjgFzpcTvmVf69GwR,2018-08-10,0.67,0.709,8,-3.921,0,0.0406,0.0169,0.00063,0.0542,0.577,98.954,4
Next music:
5,0wuy2BYIVLbflFDqnR9Jay,Orsay,nan,6,41

Next music:
44,6LpOROvVisSMcm2htQD8yP,Artificial Thrills,australian indie,27,208621,vogue bazaar,3txoXRwrXEb6OcwASZwIyS,2018-07-24,0.842,0.389,8,-17.579,0,0.0614,0.138,0.000251,0.116,0.298,105.023,4
Next music:
45,5Aad45vdhiyk8UE9YpSTP6,Be My Girl,nan,4,147043,onzekere guy,3a5u8lJqeBPWSc1jCWfhbF,2018-08-03,0.679,0.624,7,-6.126,0,0.058,0.704,0.00364,0.106,0.369,121.102,4
Next music:
46,79bPObOWqc0XqhUoE0jyLI,Bag Away,art pop,0,329013,Harriet Brown,0HSqzRnAQMXxNQczzjopsG,2018-07-27,0.797,0.738,0,-6.299,1,0.106,0.0334,0.0767,0.112,0.95,158.036,4
Next music:
47,530xIOHukf7ISm0XoorkTv,Fade Away,new jersey rap,4,193515,Internal Quest,5i1LvPPKJPoBTSk0i8gRrD,2018-07-23,0.751,0.758,0,-7.436,1,0.338,0.365,0.0,0.111,0.648,174.02,4
Next music:
48,7pmOnUw96WpDqNiIi65Pkk,WFM,footwork,1,183480,Heavee,Gant-Man,DJ Phil,Sirr TMO,3bTrwZAKTLYI9zozCH6zxw,0FRf7YoRwTB4L2HVVwzks4,4L2n1xvdqgPgQjYxLHUAbG,7wMCA0Cx8O1adCSiTV1IMY,2018-07-30,0.855,0.49,6,-13.725,0,0.268,0.0724,0.184,0.0848,0.618,160.021,4
Next musi

Next music:
87,2OS0B2x6aNyPtcN3juT3Tx,Not Your Prey,boston indie,bubblegrunge,indie folk,indie pop,33,214586,Squirrel Flower,7bI1v9NGBBhq8iGfytctni,2018-07-20,0.352,0.55,2,-6.98,1,0.0317,0.27,0.00656,0.096,0.282,135.261,4
Next music:
88,4mulBtb3PtmjuUmXlN5Vr0,Vibrate,emo rap,28,196414,6host,5GP0ySZUhhFtH62pKtZJox,2018-07-09,0.248,0.674,1,-2.765,1,0.0719,0.0593,0.0,0.0678,0.207,68.661,5
Next music:
89,5GFvl7wTZDzkFmXxjTO2uu,Dolla Bill,nan,5,237913,Hue,7fp00PE8I4TOoNDzenfi5F,2018-07-10,0.636,0.714,3,-12.587,1,0.113,0.0116,0.0,0.366,0.796,114.866,4
Next music:
90,3xGUnyCvQW1Hg0bZkdcoeK,Feel Like That,indie hip hop,10,156000,YTK,4QsNZ1JTLzhVSAzMxjLe7b,2018-07-13,0.666,0.441,2,-10.469,1,0.362,0.045,0.0,0.0551,0.424,80.109,4
Next music:
91,3jij0D3bHcoriIvIgCdN2f,the lounge,nan,0,224665,Synthe,JASPR,4RSl14NgijWIXQJTl9HAj4,0sCD2Ix7qcxwGhyFA0zEdG,2018-07-15,0.559,0.655,3,-6.586,0,0.44,0.69,0.00741,0.163,0.444,113.057,4
Next music:
92,4n6c1QwIYishGlPLG3gy1T,Pose,deep liquid bass,halftime dnb,liq

Next music:
132,7DBcFt6GCDL49KjGbCdHPz,Too Pretty,nan,6,194152,Aysha Monet,3jS1qFuvkhaMC06NZQtNNI,2018-06-15,0.901,0.534,7,-7.925,1,0.0718,0.0242,0.000424,0.384,0.228,119.98,4
Next music:
133,2c0XmymOB1Gvi3qCIB6hjO,BAD LOVE,slow game,0,201093,Gian Camp,Don Jon Taylor,lub x tpf,2qH1ZEo85DJjfg1rznRK5e,7mYusPDevQGDR48MEanhz7,5avWuziqELnzbnqCQ2RRHX,2018-06-14,0.82,0.699,9,-5.696,0,0.0394,0.435,0.0,0.317,0.481,128.028,4
Next music:
134,0UzC7cT5705XbVevagvAGK,Gold Mint,nan,3,176326,Best Picture,Bernard Jabs,1E6f2cQiLomrBegxciK0Mk,1EBoLDWzc0lpDDIVasJhAv,2018-05-22,0.692,0.551,2,-12.342,1,0.134,0.0155,0.0,0.113,0.613,146.994,4
Next music:
135,2rPgEANeTrxDTLPVw4RVoI,8-Bit Bros.,rap mocambicana,0,150557,$orr¥,M8NT BLANC,7jpWrYvGqakoZ6jIJDYyhW,6f3znQPGFBtiZM4xPVAB7l,2018-06-20,0.711,0.829,9,-4.859,0,0.0975,0.14,0.00129,0.0929,0.646,147.995,4
Next music:
136,1r96qH2Glnd0RKzqB7I8Zv,Ugly,nan,18,117692,HolyGxd,19Iazgb6MLtZg1whJiNJao,2018-06-19,0.92,0.53,0,-5.951,1,0.382,0.261,0.0,0.143,0.703,143.007,

Next music:
174,4Cl5G0xCVJjkLu18so18Yk,Beni Lane,nan,0,276363,iZem,2dJpwMtmOBr9ad4D7vRf2r,2018-06-15,0.794,0.553,9,-10.702,0,0.0899,0.000308,0.751,0.0929,0.857,131.992,4
Next music:
175,6LxKsXY4KJZKDjyUAriy3a,Heavy Petting,danish modern jazz,0,169031,Shatter Hands,1QYYoqc9d6sZPW5AtFY9Yc,2018-06-11,0.38,0.514,8,-11.586,0,0.0441,0.135,0.844,0.106,0.0374,89.953,4
Next music:
176,52LD1G7fuLyRa5R56vc0pA,Best For You,chill pop,indie r&b,0,202327,TARI,Lani Rose,2IkXfRlqGZgE7rvDkp30wf,7bdI7Q9ffxcej6cfVUktvb,2018-05-18,0.523,0.697,5,-6.877,1,0.151,0.0491,0.000126,0.137,0.33,104.326,4
Next music:
177,5qcwzdhDQjHjmu12gOLjY0,You,alternative r&b,indie soul,vapor soul,57,310597,Mk.gee,7tr9pbgNEKtG0GQTKe08Tz,2018-05-18,0.808,0.657,5,-6.67,1,0.298,0.526,0.00444,0.075,0.786,88.932,4
Next music:
178,35MYq2EVWYA47RjTeaUUgQ,She Told Me,nan,5,161187,Ra'Shaun,Chris LaBella,20WCeZRsLpyZ8FoyoSnpEt,4xXFNOuK9KjVkPBpYXeTk7,2018-05-11,0.626,0.593,0,-8.349,0,0.228,0.321,0.000173,0.0672,0.448,84.741,4
Next music:
1

...

Next music:
687,0hNyEPTkrBNDd7JXGeJIfI,Ascension,dancefloor dnb,drum and bass,electro house,liquid funk,0,237241,Bensley,3SfEKS85dj9v75ROsWJSiL,2018-02-09,0.614,0.98,1,-4.283,1,0.0724,0.0104,0.312,0.183,0.19,173.992,4
Next music:
688,2YZpBRyRaO7ZWUGlyEJDen,Intercontinental,canadian hip hop,23,170890,Tasha The Amazon,7J5HSblBjH8zvu96ovocNM,2018-02-07,0.771,0.763,11,-7.932,1,0.0356,0.0271,5.81e-05,0.0721,0.378,103.97,4
Next music:
689,6yVJyoTT6goRqEvSv6rWLZ,Magnified Pieces of Thermodynamics,ambient,compositional ambient,drift,drone,fourth world,new isolationism,nu age,sound art,warm drone,19,386110,Celer,Forest Management,6nDeZk1AU320DQaj8pD80O,2z0s65XFlIfrX3hB5A6YD7,2018-02-05,0.149,0.0838,5,-33.686,0,0.0375,0.975,0.987,0.0995,0.04,71.556,4
Next music:
690,40efsVI7MV6UY8KJQN4H1U,My Own,nan,4,220386,Nu Lighter,7fnVqEZ5dpI9wV0Z0eb9Wm,2018-01-15,0.613,0.382,2,-10.933,1,0.232,0.0307,0.0,0.193,0.656,109.824,3
Next music:
691,2DkzhstjFlsdSFXF5nupOd,Sad,downtempo bass,34,192000,Saint Sinner,T

Next music:
728,5s7zeAmulFLCim18KW8agV,Help,scandipop,17,189928,Fred Nicolas,09FoDpXlF6jkmrKi6fJa2t,2017-11-17,0.65,0.474,10,-6.27,1,0.0646,0.141,0.0,0.564,0.361,136.077,4
Next music:
729,02zNm05FD1zKFdK0bvdOCd,Alpha Go,ambeat,7,222346,The Shanghai Restoration Project,0EP5EpsiMP8oLYy7sPHwf9,2017-11-17,0.561,0.846,0,-7.284,0,0.0279,0.721,0.963,0.234,0.767,122.998,4
Next music:
730,2RPRNfoytn02Cxh3WldSgW,Ordinary Thoughts,emo,emo punk,future bass,vapor twitch,0,208941,Knapsack,Slyleaf,1ONhS8pN5UbzqoZlLF69kI,3urhrgG6tVFVPIc8Dgf3Y2,2017-11-14,0.77,0.817,4,-5.28,0,0.0784,0.0957,0.0983,0.198,0.453,170.022,4
Next music:
731,2lbXgRoYddgThTnPAbnXYJ,Dreams and Other Drugs.,nan,15,89208,Gnavis,5GArPXRmy5WUjYkkc0KVbp,2017-11-16,0.675,0.813,4,-11.107,0,0.078,0.844,0.905,0.482,0.562,140.101,4
Next music:
732,1ZGvS5RYbOfe6aV80K8PxM,for ever,nan,0,214000,land_scape,6mNqWlJjeC2M3FSLiqeuxV,2017-11-10,0.159,0.244,7,-22.199,0,0.0416,0.97,0.404,0.114,0.257,135.191,5
Next music:
733,3N45JybKHjplsIalHh4cst,H

Next music:
772,38tyozwEeWpWa2Z7bnRXQW,Midnight,brooklyn indie,31,202631,Emmett Kai,3iFMfPvVhkPDaR6uCaBV9q,2017-10-24,0.831,0.458,11,-8.585,1,0.0416,0.129,0.49,0.405,0.426,114.026,4
Next music:
773,6CJp9YwYEGnri7NgnQsrut,Love Is Overrated,nan,0,276310,Dax,5t4eU2GW1HoM1ditytx0eS,2017-10-27,0.382,0.372,11,-9.764,0,0.0378,0.897,0.000603,0.143,0.225,115.322,1
Next music:
774,4pAcD7T540cRmPhpWlYcbQ,Song About A Dream,uk house,10,403922,Al Zanders,4udfBn2nJQ6bHoMrr2lKpz,2017-10-27,0.664,0.778,8,-9.757,0,0.0703,0.403,0.00197,0.0993,0.32,124.513,4
Next music:
775,5Ywfhy1QaKiWhRB2FVJBhm,A Different Happiness,nan,0,411120,Stefano Ritteri,6mn2bA8ndJUuJVJE8WMvbK,2017-11-03,0.745,0.886,11,-10.166,0,0.0421,0.12,0.86,0.0698,0.454,121.021,4
Next music:
776,5OnAqnPnW4ME8BUMW8ZRcw,Praying Mantis,uk experimental electronic,weightless,8,268132,Walton,542yacXt2x9eLr9g8v0X8k,2017-11-10,0.793,0.615,11,-8.577,1,0.071,0.00265,0.906,0.12,0.15,132.001,4
Next music:
777,3ph4XP0xAkYkCZAGvOM9BG,Verso,ambient idm,0,

Next music:
815,4MIZw6OSk2BI95wIpErb71,Problem (Just for Today),nan,0,159724,Electric Company,4mTX3X3aLW08gv7UorzOoc,2017-10-20,0.585,0.615,11,-6.368,0,0.0331,0.000327,0.0,0.058,0.559,128.962,4
Next music:
816,3M8bjsd55Hpj0zIzf7Ez5R,Just Stay,nan,0,174718,Winter & Trabants,5EJFqkksTIGXfDEUPNXXfC,2017-10-27,0.357,0.875,2,-6.285,1,0.0399,0.0117,0.778,0.0933,0.776,152.218,4
Next music:
817,4dseqO3DfsBsGeH8uBUC7b,Waste,nan,10,221046,Cassidy Shooster,495xnv1o0ntnQx9SKhTVgt,2017-10-13,0.702,0.27,10,-9.561,0,0.0341,0.205,1.21e-05,0.11,0.136,100.023,4
Next music:
818,6spL6oedXIkDCpj7ON4DWZ,Is This Real,chillhop,16,215500,Pablo Queu,Oliver Han,6unlzVywR1plttUTcFVltU,3RDDl7EdvH4SS4KMmqjfqb,2017-10-13,0.742,0.635,7,-9.168,0,0.031,0.00904,4.44e-05,0.102,0.526,112.047,4
Next music:
819,5pJTKVSfpX1rJtTpXrpBKr,Hush (Like That) [feat. Peter Kuli],lo-fi rap,22,163656,Atwood,Peter Kuli,1dtZllLT0EINXgSftEnOjv,5lVUEGHUyXz8X6h58fuEwV,2017-10-19,0.831,0.525,5,-10.135,0,0.398,0.298,0.0,0.356,0.918,106.09,4
N

KeyboardInterrupt: 

# After you run the real time data producer, you have to go into Docker Desktop, access the Kafka container and run the command below to start the data streaming consumer that will be consumed in the recommender system:
### kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic playlist --from-beginning